# LOICA Enzymes (in branch Consortia) demonstration

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

This notebook shows the novel classes Degrader and Producer, which belong to class Impactor that represents enzyme active sites. These new features are part of git branch Consortia.

In [ ]:
def growth_rate(t):
    return 0 #lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return 1 #lc.gompertz(t, 0.01, 1, 1, 1)

## Degrader

Degrader represents an active site which degrades other GeneProducts.

First, create metabolism and genetic network, then create strain with these parameters.

In [ ]:
metab1 = lc.SimulatedMetabolism("Simulated metabolism 1", biomass, growth_rate)
genetic_n1 = lc.GeneticNetwork()
test_strain_1= lc.Strain('Test strain', genetic_n1, metab1)

Degrader takes parameters:
- name (use enzyme function name)
- enzyme (GeneProduct which has the active site)
- substrate (GeneProduct(s) which is(/are) degraded)
- Km (Michaelis constant - units should be the same as units of GeneProduct concentration. By default units are M)
- k2 (catalytic constant or rate of substrate degradation when it is in enzyme-substrate complex).

The degradation rate is calculated according to Michaelis-Menten rate law, which assumes that Enzyme-Substrate complex concentration change is in quasi-steady-state.

All concentrations are in M.
In <div class="csl-entry">Momb, J., Wang, C., Liu, D., Thomas, P. W., Petsko, G. A., Guo, H., Ringe, D., &#38; Fast, W. (2008). Mechanism of the Quorum-Quenching Lactonase (AiiA) from Bacillus thuringiensis. 2. Substrate Modeling and Active Site Mutations. <i>Biochemistry</i>, <i>47</i>(29), 7715. https://doi.org/10.1021/BI8003704</div>

wildtype lactonase Km is 5.6 mM (mmol/L). This is 0.0056 M

In [ ]:
# Create and add reporter to genetic network
c14 = lc.Regulator(name='C14', color='green', degradation_rate=0, init_concentration=5)
aiia = lc.Regulator(name='AiiA', init_concentration=0.01)
genetic_n1.add_regulator([c14, aiia])

# Degradation of c14 by lactonase
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=c14, Km=0.0056, k2=91)
genetic_n1.add_impactor(lactonase)

In [ ]:
sample1 = lc.Sample(strain=test_strain_1)
assay1 = lc.Assay([sample1], 
                n_measurements=100, 
                interval=0.24,
                name='Degradation of C14',
                description='Simulated C14 degradation by lactonase AiiA'
                )

In [ ]:
assay1.run(mode='track_all')

In [ ]:
m = assay1.measurements
fig,ax = plt.subplots()
m[m.Signal=='AiiA in Test strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
ax.legend(['AiiA'])
fig1,ax1 = plt.subplots()
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax1)
ax1.legend(['C14'])

## Two competing substrates

Degrader can take a list of GeneProducts as a substrate parameter. These substrates compete to bind to Degrader. Km and k2 values can also be supplied as a list to reflect parameters used for each substrate.

In [ ]:
metab1 = lc.SimulatedMetabolism("Simulated metabolism 1", biomass, growth_rate)
genetic_n1 = lc.GeneticNetwork()
test_strain_1= lc.Strain('Test strain', genetic_n1, metab1)

In [ ]:
# Create and add reporter to genetic network
c14 = lc.Regulator(name='C14', color='green', degradation_rate=0, init_concentration=5)
c4 = lc.Regulator(name='C4', color='blue', degradation_rate=0, init_concentration=5)
aiia = lc.Regulator(name='AiiA', init_concentration=0.01)
genetic_n1.add_regulator([c14, c4, aiia])

# Degradation of C14 and C4 by lactonase
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=[c14, c4], Km=[0.0056, 0.0028], k2=[91, 182])
genetic_n1.add_impactor(lactonase)

In [ ]:
sample1 = lc.Sample(strain=test_strain_1)
assay1 = lc.Assay([sample1], 
                n_measurements=100, 
                interval=0.24,
                name='Degradation of C14 and C4',
                description='Simulated C14 and C4 degradation by lactonase AiiA'
                )

In [ ]:
assay1.run(mode='track_all')

In [ ]:
m = assay1.measurements
fig,ax = plt.subplots()
m[m.Signal=='AiiA in Test strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
ax.legend(['AiiA'])
fig1,ax1 = plt.subplots()
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax1)
m[m.Signal=='C4 in Test strain'].plot(x='Time', y='Measurement', color='blue', style='-', ax=ax1)
ax1.legend(['C14', 'C4'])

## Producer

First, create metabolism and genetic network, then create strain with these parameters.

In [ ]:
metab2 = lc.SimulatedMetabolism("Simulated metabolism 2", biomass, growth_rate)
genetic_n2 = lc.GeneticNetwork()
test_strain_2= lc.Strain('Test strain', genetic_n2, metab2)

Producer parameters are:
- name (use enzyme function name)
- enzyme (GeneProduct which has the active site)
- product (GeneProduct which is synthesized by the enzyme) 
- k2 (catalytic constant or product synthesis rate per 1M of enzyme).

Producer class assumes that there is always enough substrate in the system, so all active sites are occupied with substrate at every moment. Because of that, only k2 parameter is needed.

In [ ]:
# Create and add reporter to genetic network
c4 = lc.Regulator(name='C4')
rhli = lc.Regulator(name='RhlI', init_concentration=5)
genetic_n2.add_regulator([c4, rhli])

# Synthesis of C4 by RhlI
c4_synthase = lc.Producer(name='C4-HSL synthase', enzyme=rhli, product=c4, k2=960)
genetic_n2.add_impactor(c4_synthase)

I have calculated k2 by using data from supplementary text from <div class="csl-entry">Pai, A., &#38; You, L. (2009). Optimal tuning of bacterial sensing potential. <i>Molecular Systems Biology</i>, <i>5</i>, 286. https://doi.org/10.1038/MSB.2009.43</div>

I used 16 mol of C4 produced per mol of RhlI per minute, multiplied it by 60 to get measurement per hour.

In [ ]:
sample2 = lc.Sample(strain=test_strain_2)
assay2= lc.Assay([sample2], 
                n_measurements=100, 
                interval=0.24,
                name='Synthesis of C4',
                description='Simulated C4 synthesis by RhlI'
                )

In [ ]:
assay2.run(mode='track_all')

In [ ]:
m = assay2.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='C4 in Test strain'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax)
m[m.Signal=='RhlI in Test strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
plt.legend(['C4', 'RhlI'])
